## <center>SQL</center>

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато теперь они могут уделять больше времени книгам. На это обратили внимание стартаперы, которые бросились создавать новые приложения для тех, кто любит читать.

Вам передали базу данных одного из конкурирующих сервисов. В ней есть информация о книгах, издательствах, авторах, а также пользовательские обзоры на книги. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

### Цели исследования

- Посчитать количество книг, выпущенных после 1 января 2000 года;
- Посчитать количество пользовательских обзоров и среднюю оценку для каждой книги;
- Определить издательство, которое издало наибольшее число книг толще 50 страниц (чтобы исключить из анализа различные брошюры);
- Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более пользовательскими оценками;
- Посчитать среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок.

**Импорт библиотек и подключение к базе**

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Выведем первые 5 строк каждой таблицы для ознакомления с данными:**

In [3]:
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']
def table_head(table):
    query = f'''
    select 
        *
    from 
        {table}
    LIMIT 5;
    '''
    df = pd.io.sql.read_sql(query, con = engine)
    
    query = f'''
    select 
        count(*) as count
    from 
        {table};
    '''
    count = pd.io.sql.read_sql(query, con = engine)
    count = count['count'][0]
    print(f'Table {table}:')
    display(df)
    print(f'В таблице {table} {count} записей')
    print()

In [4]:
for i in tables:
    table_head(i)

Table books:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


В таблице books 1000 записей

Table authors:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


В таблице authors 636 записей

Table publishers:


...

В таблице publishers 340 записей

Table ratings:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


В таблице ratings 6456 записей

Table reviews:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


В таблице reviews 2793 записей



Перед нами 5 таблиц с информацией о книгах с датой публикации, числом страниц названием, авторами, рейтингами, отзывами и т д.

**1. Посчитаем количество книг, выпущенных после 1 января 2000 года:**

In [5]:
query = '''
select 
    count(book_id) AS count
from 
    books
where
    publication_date > '2000-01-01';
'''
books_count = pd.io.sql.read_sql(query, con = engine)

In [6]:
print('Число книг, выпущенное после 1 января, 2000 года:', books_count['count'][0])

Число книг, выпущенное после 1 января, 2000 года: 819


**2. Посчитаем количество пользовательских обзоров и среднюю оценку для каждой книги:**

In [7]:
query = '''
select
    book_id,
    books.title,
    ratings.mean_rating,
    reviews.count_reviews
from
    books
left join
    (select 
        reviews.book_id as revb_id,
        count(text) as count_reviews
    from 
        reviews
    group by
        book_id) reviews on reviews.revb_id = books.book_id
left join
    (select 
        ratings.book_id as rateb_id,
        round(avg(ratings.rating :: integer), 2) as mean_rating
    from 
        ratings
    group by
        book_id) ratings on ratings.rateb_id = books.book_id;
'''
reviews_rating = pd.io.sql.read_sql(query, con = engine)
display(reviews_rating)

,book_id,title,mean_rating,count_reviews
0,652,The Body in the Library (Miss Marple #3),4.50,2.0
1,273,Galápagos,4.50,2.0
2,51,A Tree Grows in Brooklyn,4.25,5.0
3,951,Undaunted Courage: The Pioneering First Missio...,4.00,2.0
4,839,The Prophet,4.29,4.0
...,...,...,...,...
995,64,Alice in Wonderland,4.23,4.0
996,55,A Woman of Substance (Emma Harte Saga #1),5.00,2.0
997,148,Christine,3.43,3.0
998,790,The Magicians' Guild (Black Magician Trilogy #1),3.50,2.0


Таким образом удалось поулчить таблицу с количеством отзывов по каждой книге и её средним рейтингом.

**3. Определим издательство, которое издало наибольшее число книг толще 50 страниц**

In [8]:
query = '''
select
    *
from
    (select 
        publishers.publisher as publisher,
        count(books.num_pages) as count
    from 
        books
    left join publishers on publishers.publisher_id = books.publisher_id
    where
        num_pages > 50
    group by 
        publishers.publisher) as sub
order by
    sub.count desc
limit 1;
'''
publisher = pd.io.sql.read_sql(query, con = engine)
display(publisher)

,publisher,count
0,Penguin Books,42


Издательство "Penguin Books" выпустило книг, с числом страниц толще 50, больше остальных - 42 книги.

**4. Определим автора с самой высокой средней оценкой книг — будем учитывать только книги с 50 и более пользовательскими оценками**

In [9]:
query = '''
select
    author,
    avg(mean_rating) as mean_rating
from
    (select
        books.title,
        authors.author,
        count(ratings.rating) as count_marks,
        round(avg(ratings.rating), 2) as mean_rating
    from
        books
    left join authors on authors.author_id = books.author_id
    left join ratings on ratings.book_id = books.book_id
    group by
        books.title,
        author) as sub
where
    sub.count_marks > 50
group by 
sub.author
order by
mean_rating desc
limit 1;
'''
author = pd.io.sql.read_sql(query, con = engine)
display(author)

,author,mean_rating
0,J.K. Rowling/Mary GrandPré,4.285


Из списка книг с более чем 50-ю оценками самый высокий средний рейтинг у J.K. Rowling, который составляет - 4.285.

**5. Посчитаем среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок**

In [10]:
query = '''
select
    round(avg(sub2.count_reviews), 2) as average_num_of_reviews
from
    (select
        sub.rate_username,
        count(sub.text) as count_reviews,
        count(sub.rating) as count_marks
    from
        (select 
            ratings.username as rate_username,
            reviews.text as text,
            ratings.rating as rating,
            reviews.username as rev_username
        from 
            ratings
        left join reviews on reviews.book_id = ratings.book_id and reviews.username = ratings.username)as sub
    group by
        sub.rate_username
    having
    count(sub.rating) > 50) as sub2;

'''
review_num = pd.io.sql.read_sql(query, con = engine)
display(review_num)

,average_num_of_reviews
0,24.33


Среднее количество текстовых обзоров на пользователя, который поставил более 50 оценок составляет 24